In [2]:
from transformers import AutoTokenizer
classification = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(classification)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
def tokenizerFunction(text):
    return tokenizer(text["text"], truncation=True , padding=True , max_length=128)
tokenized_dataset  = dataset.map(tokenizerFunction, batched=True )

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [29]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [36]:
training_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(3000))
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(3000))

In [7]:
from transformers import AutoModelForSequenceClassification , Trainer
model = AutoModelForSequenceClassification.from_pretrained(classification , num_labels=5)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [11]:
import evaluate
import  numpy as np
metrices = evaluate.load("accuracy")

In [23]:
def compute_metrices(eval_pred):
    logits , labels = eval_pred
    predcitions = np.argmax(logits,axis=-1)
    return metrices.compute(predictions=predcitions,references=labels)

In [13]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [14]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 1,841,157 || all params: 357,206,026 || trainable%: 0.5154


In [15]:
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="/content/",
    logging_dir="/content/",
    report_to="wandb",
    eval_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    logging_steps=10,

)

In [37]:
trainer = Trainer(model=lora_model , args=args, compute_metrics=compute_metrices, train_dataset = training_dataset , eval_dataset = eval_dataset)
lora_model.save_pretrained("/content/")

In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.859100,0.921314,0.603000
2,0.782800,0.889678,0.610000
3,0.685800,0.885008,0.620333
4,0.749600,0.901255,0.614667
5,0.830000,0.882466,0.619333


TrainOutput(global_step=940, training_loss=0.8291110723576647, metrics={'train_runtime': 1064.2618, 'train_samples_per_second': 14.094, 'train_steps_per_second': 0.883, 'total_flos': 3515988165120000.0, 'train_loss': 0.8291110723576647, 'epoch': 5.0})

In [41]:
from peft import PeftModel
lora_model = PeftModel.from_pretrained(model ,"/content/checkpoint-500")

merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained("./merged_model")



In [43]:
import numpy as np
import torch

In [59]:
from transformers import AutoModelForSequenceClassification

# Load the merged model
model = AutoModelForSequenceClassification.from_pretrained("/content/merged_model")
# Perform inference as usual
inputss =["Wow This is amazing",
         "I dont like this atall",
         "i like it but not too much",
          ]
for element in inputss:
  inputs = tokenizer(element, return_tensors="pt")
  torch.no_grad()
  outputs = model(**inputs)
  prediction = torch.nn.functional.softmax(outputs.logits,dim =-1)
  predicted_class = torch.argmax(prediction , dim=1)
  sequence_label = ["1 Star","2 Star","3 Star","4 Star","5 Star"]
  print(f"Text : {element}\n Rating: {sequence_label[predicted_class]}")


Text : Wow This is amazing
 Rating: 5 Star
Text : I dont like this atall
 Rating: 1 Star
Text : i like it but not too much
 Rating: 3 Star
